# GenAI Intensive Course Capstone Project

## Overview

Reading the long and complicated terms of service on websites can be overwhelming for most people. These documents are full of legal language that many users don't read or fully understand, which can lead to uncertainty about the specifics of the service they are using, what they are agreeing to, and the responsibilities of the provider. This project tackles that problem by creating a smart "Terms of Service Translator." It uses Gemini AI to turn complex legal text into simple, easy-to-understand summaries. The translated texts are written in plain English and made more approachable with helpful emoji.

To demonstrate this project, I have written a sample terms of service document for a fictitious company called "Dig-A-Hole." This company provides a service where customers can engage in the activity of digging holes in designated areas of a field for recreational exercise and stress relief, with options for subscription. This sample terms of service document has been saved in a PDF file, which Gemini AI is then used to read and process. The goal is to translate the formal legal language of this document into a clear and friendly summary.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample-terms-of-service/sample_terms_of_service.pdf


## Install the Python SDK

In [2]:
!pip install -Uq "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.1 MB/s eta 0:00:00


In [3]:
from google import genai
from google.genai import types

from IPython.display import Markdown, display

genai.__version__

'1.7.0'

## Set up the API key

In [4]:
from kaggle_secrets import UserSecretsClient

client = genai.Client(api_key=UserSecretsClient().get_secret("GOOGLE_API_KEY"))

## Automated retry

This codelab sends a lot of requests, so set up an automatic retry that ensures your requests are retried when per-minute quota is reached.


In [5]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## Model Selection

In [6]:
for model in client.models.list():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01

For summarizing the main document, I'm rolling with **gemini-1.5-pro**. It seems pretty reliable and stable for text-based tasks like this.

(Quick heads-up though! For the evaluation part later on, I had to switch gears and use **gemini-2.0-flash**.)

In [7]:
from pprint import pprint

for model in client.models.list():
  if model.name == 'models/gemini-1.5-pro':
    pprint(model.to_json_dict())
    break

{'description': 'Stable version of Gemini 1.5 Pro, our mid-size multimodal '
                'model that supports up to 2 million tokens, released in May '
                'of 2024.',
 'display_name': 'Gemini 1.5 Pro',
 'input_token_limit': 2000000,
 'name': 'models/gemini-1.5-pro',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countTokens'],
 'tuned_model_info': {},
 'version': '001'}


## Document Understanding: Cracking Open the PDF! 🔓

So, I've got my super important (but kinda snooze-worthy) terms of service all written down and tucked away in a PDF file. Right here, I'm gonna pull that document in and have Gemini work its magic to give me
a quick and easy rundown in plain English – no legal headaches! 🥳

In [8]:
document_file = client.files.upload(file='/kaggle/input/sample-terms-of-service/sample_terms_of_service.pdf')

In [9]:
request = 'Please summarize the following terms of servicein plain, easy-to-undersand English:'

def translate_doc(request: str) -> str:
  """Execute the request on the uploaded document."""
  # Set the temperature low to stabilise the output.
  config = types.GenerateContentConfig(temperature=0.0)
  response = client.models.generate_content(
      model='gemini-1.5-pro',
      config=config,
      contents=[request, document_file],
  )

  return response.text

summary = translate_doc(request)
Markdown(summary)

This is a summary of the Dig-A-Hole Terms of Service:

**What Dig-A-Hole offers:**  Dig-A-Hole lets you dig holes in their field for fun and stress relief. They offer a free trial dig, monthly subscriptions, and optional shovel rentals.

**Agreement:** By using Dig-A-Hole's services, you agree to these terms.

**Subscriptions:** Subscriptions are for a minimum of one month.  You'll be charged monthly, starting on the day you subscribe. You can cancel anytime, but you still have to pay for the current month. No refunds for partial months.

**Shovel Rental:** You can rent shovels for an extra fee. You're responsible for returning them in good condition (normal wear and tear is okay). Dig-A-Hole can charge you for lost or damaged shovels.

**Safety and Risks:** Digging has risks (like injuries). You accept these risks and are responsible for your own safety.

**Liability:** You agree not to sue Dig-A-Hole for anything that happens while you're digging, even if you get hurt. You also agree to protect Dig-A-Hole from any legal claims related to your use of their services.

**Medical Disclaimer:** Dig-A-Hole isn't responsible for your health. See a doctor before digging, especially if you have health problems.

**Conduct:** Be respectful and responsible. Dig-A-Hole can refuse service to anyone who isn't.

**Changes to Terms:** Dig-A-Hole can change these terms anytime.  Using their services after a change means you accept the new terms.

**Law and Disputes:** Ohio law governs these terms. Any disputes will be handled in Columbus, Ohio courts.

**Contact:** Contact information is provided for questions.


## Few-Shot Prompt: Making it Friendly! 🎉

The summary I got above is okay, but let's dial up the friendliness! To get Gemini to be a bit more casual and throw in some fun emoji, I am going to use a "few-shot prompt." Think of it as showing Gemini a few examples of how I want the translations to sound – super chill and emoji-packed! 😎

In [10]:
model_config = types.GenerateContentConfig(
    temperature=0.0
)

few_shot_prompt = """
Here are some examples of how to translate leagal terms of service and their friendly summaries using emoji:

Original: "We reserve the right to accept or refuse membership in our discretion." 
Translated: "We get to say yay 👍 or nay 👎 to memberships, just because we can💪!"

Original: "To the maximum extent permitted by law, you agree that the Company shall not be held liable for any injuries, damages, or losses incurred in connection with the use of our services or products. By using our services, you waive any right to bring a claim or lawsuit against us for such injuries."
Translated: "Oops! If you trip, slip, or fall dramatically and get hurt while using our stuff, please don’t sue us 😅🙏. By hanging out with us, you're saying, 'Okay cool, I won't blame you if I bonk myself💖'"

Please summarize the following terms of servicein plain, easy-to-undersand English with emoji.
"""

response = client.models.generate_content(
    model='gemini-1.5-pro',
    config=model_config,
    contents=[few_shot_prompt, document_file],
)

print(response.text)

Welcome to Dig-A-Hole! 🕳️  Here's the simple version of our rules:

* **What we do:** We give you a place to dig holes!  It's for fun and stress relief. 😄  We have a field with designated digging areas.
* **Free Trial:** First dig is free! 🎉
* **Membership:**  You can sign up for a monthly membership to keep digging. 🗓️  One month minimum.
* **Shovels:** We rent shovels 🥄 for an extra fee (if available).  Please return them in good shape (normal wear and tear is okay).
* **Safety First!** ⚠️ Digging can be risky (sprains, strains, trips, falls, etc.). You're responsible for your own safety.  Be careful!
* **No Lawsuits Please!** 🙏  If you get hurt, you can't sue us.  By digging here, you agree to this.
* **Be Nice!**  Be respectful to others.  We can refuse service if you're disruptive or unsafe. 😠
* **We Can Change the Rules:** We might update these terms.  Using our services after changes means you agree to the new rules.
* **Questions?**  Contact us! 📞 📧
* **Ohio Law Rules:**  Ohio 

## JSON: Keeping Things Linked

Good summaries so far! Now, let's structure things with JSON to keep each "Original" legal term directly linked to its "Translated" buddy. 

In [11]:
import json

few_shot_prompt = """
Please translate the following terms of service to more casual,friendly, easy-to-understand English with emoji. The output should be a JSON array and each element is
a JSON object with the "Original" and "Translated" version.
Here are some examples of how to translate leagal terms of service and their friendly summaries using emoji:
```
{
Original: "We reserve the right to accept or refuse membership in our discretion." 
Translated: "We get to say yay 👍 or nay 👎 to memberships, just because we can💪!"
}
{
Original: "To the maximum extent permitted by law, you agree that the Company shall not be held liable for any injuries, damages, or losses incurred in connection with the use of our services or products. By using our services, you waive any right to bring a claim or lawsuit against us for such injuries."
Translated: "Oops!😳 If you trip, slip, or fall dramatically and get hurt🤕 while using our stuff, please don’t sue us 😅🙏. By hanging out with us, you're saying, 'Okay cool, I won't blame you if I bonk myself💖'"
}
```

"""

response = client.models.generate_content(
    model='gemini-1.5-pro',
    contents= [few_shot_prompt, document_file],
    config={
        'response_mime_type': 'application/json'
    },
)
     
#print(response.text)

response_load = json.loads(response.text)
print(json.dumps(response_load, indent=4, ensure_ascii=False))

[
    {
        "Original": "We reserve the right to accept or refuse membership in our discretion.",
        "Translated": "We can choose to accept 👍 or deny 👎 memberships as we see fit."
    },
    {
        "Original": "To the maximum extent permitted by law, you agree that the Company shall not be held liable for any injuries, damages, or losses incurred in connection with the use of our services or products. By using our services, you waive any right to bring a claim or lawsuit against us for such injuries.",
        "Translated": "Legally speaking, we aren't responsible if you get hurt🤕 using our services. By using our services, you agree not to sue us. 🙏"
    },
    {
        "Original": "These Terms of Service constitute a legally binding agreement between you (\"Customer,\" \"you,\" or \"your\") and Dig-A-Hole (\"Dig-A-Hole,\" \"we,\" \"us,\" or \"our\").",
        "Translated": "This is a legal agreement between you and us (Dig-A-Hole)🤝."
    },
    {
        "Original": "By 

In [12]:
#I've got my data all nicely packed in JSON. But what if I want to parse it?  
#Here's how I do the parsed summary.

import json
import re

try:
    print("\nParsed JSON Summary:")

    if isinstance(response_load, list):
        for i, item in enumerate(response_load, start=1):
            original = item.get('Original', '[Missing original]')
            translated = item.get('Translated', '[Missing translated]')
            print(f"\nItem {i}:")
            print(f"  Original: {original}")
            print(f"  Translated: {translated}")
    else:
        print("Expected a JSON array but received a different structure.")
except json.JSONDecodeError as e:
    print("\nError decoding JSON:")
    print(f"  {e}")
    print("The output does not appear to be valid JSON.")


Parsed JSON Summary:

Item 1:
  Original: We reserve the right to accept or refuse membership in our discretion.
  Translated: We can choose to accept 👍 or deny 👎 memberships as we see fit.

Item 2:
  Original: To the maximum extent permitted by law, you agree that the Company shall not be held liable for any injuries, damages, or losses incurred in connection with the use of our services or products. By using our services, you waive any right to bring a claim or lawsuit against us for such injuries.
  Translated: Legally speaking, we aren't responsible if you get hurt🤕 using our services. By using our services, you agree not to sue us. 🙏

Item 3:
  Original: These Terms of Service constitute a legally binding agreement between you ("Customer," "you," or "your") and Dig-A-Hole ("Dig-A-Hole," "we," "us," or "our").
  Translated: This is a legal agreement between you and us (Dig-A-Hole)🤝.

Item 4:
  Original: By using our services, you acknowledge that you have read, understood, and agr

## Define an Evaluator: Checking the Translation Quality

My friendly translation are looking good against the original text. Now, the question is: how well did my translator actually do?

In this section, I'm going to put the translations to the test by focusing on a few key things that matter:

* **Clarity:** How easy is the translated text to understand for someone without a law degree?
* **Friendliness:** Did the translation nail that approachable and emoji-filled vibe I was going for?
* **Completeness:** Did it capture all the important info from the original legal stuff?

To figure this out, I've set up a simple rating system, giving a score from 1 to 5. I've also laid out the evaluation steps. Let's see how they measure up!

**Important:** To keep things running smoothly and avoid any hiccups with rate limits, I'm only going to evaluate the first three items from the terms of service for now. But I believe this will give me a good initial idea of how well the translator is performing.

In [13]:
import enum

# Define the evaluation prompt for evaluating the "Translated" text
TRANSLATION_EVAL_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the AI-generated translation of a sentence from a terms of service
document.
We will provide you with the original sentence and the AI-generated translation.
You should first read the original sentence carefully, then evaluate the quality of the translation based on the Criteria 
provided in the Evaluation section below.
You will assign the translation a rating following the Rating Rubric and Evaluation Steps. Give step-by-step explanations 
for your rating, and only choose ratings from the Rating Rubric.

# Evaluation
## Metric Definition
You will be assessing the translation Clarity, friendliness and Completeness.

## Criteria
Clarity: The translation prioritizes making the core meaning immediately accessible and easy to understand for everyone, 
even those unfamiliar with legal terms. It uses simple language and relatable analogies to convey the essential information effectively.
Friendliness: The translation adopts an extremely approachable, casual, and enthusiastic tone, using emojis and informal language 
to create a positive and engaging experience for the reader. The high level of friendliness is intentional to make the legal terms 
feel less intimidating and more welcoming.
Completeness: Completeness: The translation accurately and fully conveys the core meaning and key pieces of information presented 
in the original sentence. It should capture all the essential facts, actions, entities, and relationships described, 
even if expressed in simpler language. The level of detail in the translation should be sufficient to understand the main points 
of the original without losing critical information.

## Rating Rubric
5: (Very good). The translation is accurate, clear, friendly, and complete.
4: (Good). The translation is mostly accurate, clear, friendly, and complete.
3: (Ok). The translation is understandable but may have minor issues with accuracy, clarity, or friendliness. Emojis might 
be missing or slightly awkward.
2: (Bad). The translation has significant issues with accuracy or clarity, or fails to adopt a friendly tone.
1: (Very bad). The translation is inaccurate, incomprehensible, or completely fails to address the original meaning.

## Evaluation Steps
STEP 1: Assess the translation in aspects of clarity, friendliness, and completeness according to the criteria.
STEP 2: Score based on the rubric.

# User Inputs and AI-generated Response
## Original Sentence

{original}

## AI-generated Translation

{response}
"""

# Define a structured enum class to capture the result.
class SummaryRating(enum.Enum):
    VERY_GOOD = '5'
    GOOD = '4'
    OK = '3'
    BAD = '2'
    VERY_BAD = '1'

def eval_translation(original, ai_response):
    """Evaluate the generated translation against the original sentence."""

    # It doesn't look like I can use chat with gemini-1.5-pro. Using gemini-2.0-flash here.
    chat = client.chats.create(model='gemini-2.0-flash')
    
    # Generate the full text response.
    response = chat.send_message(
        message=TRANSLATION_EVAL_PROMPT.format(original=original, response=ai_response)
    )
    verbose_eval = response.text
    
    # Coerce into the desired structure.
    structured_output_config = types.GenerateContentConfig(
        response_mime_type="text/x.enum",
        response_schema=SummaryRating,
    )
    response = chat.send_message(
        message="Convert the final score.",
        config=structured_output_config,
    )
    structured_eval = response.parsed
    
    return verbose_eval, structured_eval

# Ideally, I want to check everything but I don't have enough quota. So, iterating only first three elements.
evaluation_results = []
for item in response_load[:3]:
    original_text = item['Original']
    translated_text = item['Translated']
    verbose_eval, structured_eval = eval_translation(original=original_text, ai_response=translated_text)

    evaluation_results.append({
        'original': original_text,
        'translated': translated_text,
        'verbose_evaluation': verbose_eval,
        'structured_evaluation': structured_eval.name if structured_eval else None
    })

for result in evaluation_results:
    print(f"Original: {result['original']}")
    print(f"Translated: {result['translated']}")
    print(f"Verbose Evaluation:\n{result['verbose_evaluation']}")
    print(f"Structured Evaluation: {result['structured_evaluation']}")
    print("-" * 20)

Original: We reserve the right to accept or refuse membership in our discretion.
Translated: We can choose to accept 👍 or deny 👎 memberships as we see fit.
Verbose Evaluation:
STEP 1:
The translation captures the core meaning of the original sentence in a clear and straightforward way. It uses simple language and familiar terms ("accept," "deny," "as we see fit") to make the legal concept more accessible. The use of emojis adds to the friendly tone. All key information is preserved.
Clarity: The translation is very clear.
Friendliness: The translation is friendly because of the emojis.
Completeness: The translation is complete.

STEP 2:
Based on the evaluation, the translation is very good.
Rating: 5

Structured Evaluation: VERY_GOOD
--------------------
Original: To the maximum extent permitted by law, you agree that the Company shall not be held liable for any injuries, damages, or losses incurred in connection with the use of our services or products. By using our services, you waiv